This is a document I use to separate out functions of the main script in order to troubleshoot or quickly bypass some steps. I often use this to avoid spamming chat or API requests when I want to test changes in a script without using quota credits or some other resource.

In [5]:
from funcs import *
import requests,time
from pprint import pprint
showbot_channel = 'Frogpants'
with open('../showbot_token.txt',"r") as f:
    token = f.readlines()[0]
#Scott's normal channel name
channel_id = '@ScottJohnson'
#Hard-coded channel id, can be used in place of channel_id for more stability but less convenience
numeric_id = 'UCIEIRz-KpYoEPnrNQuyHwJw'

Activating the venv in VSCode can be tricky

In [8]:
! ".venv/Scripts/activate.bat"

Separated Auto detect function

In [6]:
streamid = find_stream_id('@ScottJohnson')
if streamid:
    print(f'Live stream detected.\nLive stream id: {streamid}')
else:
    print('No live stream found.')

Live stream detected.
Live stream id: fsTfWrRMDAs


In [6]:
streamid = 'fsTfWrRMDAs'

For testing chat ingestion regardless of triggering submission or not

In [ ]:
authors,messages = parse_chat(read_chat(find_chat_id(streamid)))
for author,message in zip(authors,messages):
    print(f'{author}: {message}')

For titles, testing triggers

In [ ]:
author_index,titles = title_search(read_chat(find_chat_id(streamid)))
for author,title in zip(authors,titles):
    print(f'{author}: {title}')

Transforming strings to HTML encoding

In [18]:
authors_html,titles_html = [html_ify(auth) for auth in author_index],[html_ify(title) for title in titles]
for author_html,title_html in zip(authors_html,titles_html):
    print(f'{author_html}: {title_html}')
    

Credentials testing playground

In [ ]:
#Just to test livechatid function or get it outside GUI loop
livechatid = find_chat_id(streamid)

Credentials found.


In [ ]:
#Or set it manually and save an API credit
livechatid = 'Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM'

In [ ]:
#Scrape titles from Showbot. No YouTube interaction, so not API credits will be used
#!! suspect this may be where YouTube can be overwritten with showbot if the bot gets restared mid-stream.
author_index,submitted_titles = build_submission_history(showbot_channel)

Titles scraped for Showbot.tv channel: Frogpants 


In [9]:
for author,title in zip(author_index,submitted_titles):
    print(f'{author}: {title}')

In [ ]:
credentials,status = build_credentials(client_secrets_file,scopes,testing=True)
credentials

Credentials found.
Credentials refreshed.
New credentials saved via default flow.


In [ ]:
#the flow variable is the core element of the others, so this is about the most basic unit of OAuth2 flow that one can isolate at this point.
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file,scopes)
credentials = flow.run_local_server(port=0)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=347817049340-9r6vmp3vflu05g81jt897f3hankoud92.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53833%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=b3QgYtquX0vawvInwuJGjpN4fMEOsi&access_type=offline


In [ ]:
credentials,status = build_credentials(client_secrets_file,scopes)

Credentials found.


In [17]:
youtube = build_yt_obj(credentials,api_service_name, api_version,module=googleapiclient.discovery)
youtube

In [15]:
with open('../token.json','w') as token:
    token.write(credentials.to_json())

With an OAuth2 successfully completed, one can now send any message one wants to an identified livechatid

In [ ]:
message = 'Custom message to send to livechatid'
try:
    send_chat_message('Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM',message)
except googleapiclient.errors.HttpError:
    print('No live chat detected, likely because the chat has ended.')

In [ ]:
credentials,status = build_credentials(client_secrets_file,scopes) #Builds OAuth2 credentials object. Uses existing files or makes new one if not detected. This links the function to the account that follows the URI.
youtube = build_yt_obj(credentials) #Common step in calling the API. Always have to build an object on which to call methods.

Credentials found.
Credentials rebuilt via error.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=347817049340-9r6vmp3vflu05g81jt897f3hankoud92.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=7D4cyVk2aBsqFxENfOuuWEeVnmJheF&access_type=offline
New credentials saved via function.


In [ ]:
#Intended as a flexible way to add Patreon page names and page links, but not useable unless links are allowed to be submitted via the YouTube API
message_raw = ['TMS is a collaborative effort! ']
i = 0
for patreon in patreon_links:
    i += 1
    link = patreon_links[patreon]
    if patreon == 'TMS':
        message = f'Support TMS directly at {link}, '
    elif patreon == 'Frogpants':
        message = f'or support the whole network at: {link}. Support individuals: '
    elif i == len(patreon_links):
        message = f' and {patreon} at {link}.'
    else:
        message = f' {patreon} at {link},'
    message_raw.append(message)

NameError: name 'patreon_links' is not defined

Older formats of the main showbot loop, keeping in case

In [ ]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            requests.get(submission_link)
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials,status = build_credentials(client_secrets_file,scopes) #Builds OAuth2 credentials object. Uses existing files or makes new one if not detected. This links the function to the account that follows the URI.
            youtube = build_yt_obj(googleapiclient.discovery,credentials) #Common step in calling the API. Always have to build an object on which to call methods.
            response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
            print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
    time.sleep(5)

In [ ]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            try:
                requests.get(submission_link)
            except TimeoutError:
                print('Timeout error')
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials = Credentials.from_authorized_user_file("../working_submit_token_Talia_do_not_delete.json", scopes)
            youtube = build_yt_obj(googleapiclient.discovery,credentials,api_service_name, api_version) #Common step in calling the API. Always have to build an object on which to call methods.
            try:
                response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
                print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
            except googleapiclient.errors.HttpError: #For some reason the script can get a 403 response code error when attempting to submit a confirmation response. This block keeps the script from halting if that occurs.
                print(f'HTTP Error: [{author}: {title}] failed to submit.')# Trying again in 3 seconds.') #This is an experimental approach to handling the 403 error by attempting to resubmit the response asynchronously.
                # asyncio.sleep(3)
                # try:
                #     response = asyncio.send_async_chat_message(livechatid,author)
                # except googleapiclient.errors.HttpError:
                #     print(f'Submission of [{author}: {title}] failed.')
                # print(traceback.format_exc())
    time.sleep(5) 